In [223]:
import csv
import numpy as np
import copy

In [224]:
#read csv file function
def readCSV(file_address = ""):

    # default file address
    if len(file_address) == 0:
        file_address = "F:\ML and CV\WPI_AI\Assignment_1\\board"

    board = []

    with open(file_address, mode='r',encoding='utf-8-sig')as file:
        csvFile = csv.reader(file)

        for lines in csvFile:
            board.append(list(lines))

        for rows in range(len(board)):
            print(board[rows])

    file.close()
    return board

In [225]:
grid = readCSV(file_address = r"C:\Users\Mandeep Singh\My python\Artificial Intelligence\Assignment 3\Gridworld.csv")

['3', '0', '0', 'X', '0']
['-2', '0', '0', '0', '0']
['S', '0', '0', '0', '0']


In [226]:
grid_row_size = len(grid)
grid_col_size = len(grid[0])
for i in range(0, grid_row_size):
    for j in range(0, grid_col_size): 
        if grid[i][j]=='S' or grid[i][j]== 'X':
            grid[i][j] = grid[i][j]
        else:
            grid[i][j] = int(grid[i][j])

In [227]:
#general parameters(some of them have to be input through command line)
alpha = 0.1
gamma = 0.9           #command line input required
p_up = 1           #command line input required
p_down = (1-p_up)/2
p_2up = (1-p_up)/2
R = 0             #command line input required
time = 20             #command line input required

In [228]:
#find the pits in grid
walls = []
for i in range(0,grid_row_size):
    for j in range(0,grid_col_size):
        if grid[i][j] == 'X':
            walls.append([i,j])
            
#find the start point
def current_pos(grid):
    current_position = []
    for i in range(0,grid_row_size):
        for j in range(0,grid_col_size):
            if grid[i][j] == 'S':
                current_position.append([i,j])
    return current_position

def state_reward(position):
    i = position[0]
    j = position[1]
    if grid[i][j] == 'S' or grid[i][j] == 'X':
        reward = 0
    else:
        reward = grid[i][j]
    return reward

for i in range(0,grid_row_size):
    for j in range(0,grid_col_size):
        position = [i,j]
#         print(state_reward(position))

end_states = []
for i in range(0,grid_row_size):
    for j in range(0,grid_col_size):
        if grid[i][j]!=0 and grid[i][j]!='S':
            end_states.append([i,j])

In [229]:
def takeAction(current_position, direction, walls, grid_row_size, grid_col_size):

        new_position = copy.copy(current_position)
        if direction == "up":
            action = np.random.choice(['up', '2up', 'down'], p=[p_up, p_2up, p_down])
        if direction == "down":
            action = np.random.choice(['down', '2down', 'up'], p=[p_up, p_2up, p_down])
        if direction == "left":
            action = np.random.choice(['left', '2left', 'right'], p=[p_up, p_2up, p_down])
        if direction == "right":
            action = np.random.choice(['right', '2right', 'left'], p=[p_up, p_2up, p_down])
        
        if action == 'up':
            new_position[0] += -1
        elif action == 'down':
            new_position[0] += 1
        elif action == 'right':
            new_position[1] += 1
        elif action == 'left':
            new_position[1] += -1
        elif action == '2up':
            new_position[0] += -1
            barrier = new_position in walls or new_position[0] < 0 or new_position[0] >= grid_row_size or new_position[1] < 0 or new_position[1] >= grid_col_size
            if not barrier:
                current_position = copy.copy(new_position)
                new_position[0] += -1
        elif action == '2down':
            new_position[0] += 1
            barrier = new_position in walls or new_position[0] < 0 or new_position[0] >= grid_row_size or new_position[1] < 0 or new_position[1] >= grid_col_size
            if not barrier:
                current_position = copy.copy(new_position)
                new_position[0] += 1
        elif action == '2right':
            new_position[1] += 1
            barrier = new_position in walls or new_position[0] < 0 or new_position[0] >= grid_row_size or new_position[1] < 0 or new_position[1] >= grid_col_size
            if not barrier:
                current_position = copy.copy(new_position)
                new_position[1] += 1
        elif action == '2left':
            new_position[1] += -1
            barrier = new_position in walls or new_position[0] < 0 or new_position[0] >= grid_row_size or new_position[1] < 0 or new_position[1] >= grid_col_size
            if not barrier:
                current_position = copy.copy(new_position)
                new_position[1] += -1

        barrier = new_position in walls or new_position[0] < 0 or new_position[0] >= grid_row_size or new_position[1] < 0 or new_position[1] >= grid_col_size

        if not barrier:
            current_position = new_position

        return current_position


In [230]:
class state:
    def __init__(self,position,Q_val):
        self.position = position
        self.Q_val = Q_val
        self.reward = state_reward(self.position)
        
states = []
for i in range(0,grid_row_size):
    for j in range(0,grid_col_size):
        position = [i,j]
        state_curr = state(position,[0,0,0,0])
        states.append(state_curr)
        
        
action = ["up","down", "left", "right"]

counter = 30
for j in range(0,counter):
    grid_states = copy.deepcopy(states)
    for i in range(0,len(grid_states)):
        current_state = grid_states[i]
        current_position = current_state.position
        if current_position in end_states:
#             print([i],"  ",current_state.Q_val)
            continue
        else:
            for k in range(0,4):
                next_position = takeAction(current_position, action[k], walls, grid_row_size, grid_col_size)
                for l in range(0,len(states)):
                    if next_position == states[l].position:
                        next_state = states[l]
                        if next_state.position!=current_state.position:
                            ind = next_state.Q_val.index(max(next_state.Q_val))
                            current_state.Q_val[k] = current_state.Q_val[k] + alpha*((R+next_state.reward) + gamma*next_state.Q_val[ind] - current_state.Q_val[k])
                            break
                        else:
                            continue
#     print("-------------")
#     for i in range(0,len(grid_states)):
#         print(grid_states[i].Q_val)
    states = copy.deepcopy(grid_states)
                    
# print([i],"  ",current_state.Q_val)

In [231]:
for i in range(0,len(grid_states)):
    print(grid_states[i].Q_val)

[0, 0, 0, 0]
[0, 1.430416487870402, 2.8728265251743514, 1.430416487870402]
[0, 0.7710505312912284, 2.2040234481799708, 0]
[0, 0, 0, 0]
[0, 0.1296540813271352, 0, 0]
[0, 0, 0, 0]
[2.2040234481799708, 0.7710505312912284, -1.9152176834495678, 0.7710505312912284]
[1.430416487870402, 0.3454265704024156, 1.430416487870402, 0.3454265704024156]
[0, 0.1296540813271352, 0.7710505312912284, 0.1296540813271352]
[0.04117624317982215, 0.04117624317982215, 0.3454265704024156, 0]
[-1.9152176834495678, 0, 0, 0.7710505312912284]
[1.430416487870402, 0, 0.3454265704024156, 0.3454265704024156]
[0.7710505312912284, 0, 0.7710505312912284, 0.1296540813271352]
[0.3454265704024156, 0, 0.3454265704024156, 0.04117624317982215]
[0.1296540813271352, 0, 0.1296540813271352, 0]
